# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**\
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**\
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [1]:
path = '/Users/camilledemers/Documents/03-corpus/0-listes_sites/'
acteur = "msss"

La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
On commence donc par lire le CSV pour extraire nos URLS.

In [18]:
from pandas import *

# encoding="utf-8"
with open(path + acteur + "/" + acteur + '.csv') as f:
    csv = read_csv(f, sep=";")

csv

,Address,Status-Code,Status-Text,Type,Size,Title,Date,Level,Links Out,Links In,Server,Error,Duration,Charset,Description
0,http://www.finances.gouv.qc.ca/,200,ok,text/html,31248,Ministère des finances | Page d'accueil,NaN,0,70,205,NaN,NaN,00:00.306,NaN,NaN
1,http://www.finances.gouv.qc.ca/index.asp,200,ok,text/html,31248,Ministère des finances | Page d'accueil,NaN,1,70,207,NaN,NaN,00:00.984,NaN,NaN
2,http://www.finances.gouv.qc.ca/fr/plan.asp,200,ok,text/html,18727,Ministère des Finances | Plan du site,NaN,1,124,206,NaN,NaN,00:00.952,utf-8,NaN
3,http://www.finances.gouv.qc.ca/fr/lien.asp,200,ok,text/html,14541,Ministère des Finances | Liens utiles,NaN,1,76,206,NaN,NaN,00:01.093,utf-8,NaN
4,http://www.finances.gouv.qc.ca/index_en.asp,200,ok,text/html,31601,Ministère des finances | Page d'accueil,NaN,1,63,163,NaN,NaN,00:01.047,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,http://www.finances.gouv.qc.ca/mesures_ecofisc...,200,ok,text/html,17355,Remboursement de la taxe sur les carburants à ...,NaN,4,44,1,NaN,NaN,00:03.305,NaN,NaN
231,http://www.finances.gouv.qc.ca/mesures_ecofisc...,200,ok,text/html,18008,Système de plafonnement et d’échange de droits...,NaN,4,43,1,NaN,NaN,00:03.099,NaN,NaN
232,http://www.finances.gouv.qc.ca/mesures_ecofisc...,200,ok,text/html,17759,Taxe sur les carburants | Mesures écofiscales,NaN,4,43,1,NaN,NaN,00:02.896,NaN,NaN
233,http://www.finances.gouv.qc.ca/mesures_ecofisc...,200,ok,text/html,17466,Droits annuels exigibles des titulaires d’une ...,NaN,4,43,1,NaN,NaN,00:03.648,NaN,NaN


In [19]:
liste = csv['Address'].tolist()
liste

['http://www.finances.gouv.qc.ca/',
 'http://www.finances.gouv.qc.ca/index.asp',
 'http://www.finances.gouv.qc.ca/fr/plan.asp',
 'http://www.finances.gouv.qc.ca/fr/lien.asp',
 'http://www.finances.gouv.qc.ca/index_en.asp',
 'http://www.finances.gouv.qc.ca/abonnement',
 'http://www.finances.gouv.qc.ca/fr/Ministere87.asp',
 'http://www.finances.gouv.qc.ca/fr/Ministres5.asp',
 'http://www.finances.gouv.qc.ca/fr/Budget_et_mise_a_jour.asp',
 'http://www.finances.gouv.qc.ca/fr/comm.asp',
 'http://www.finances.gouv.qc.ca/fr/RI_OV_Emissions_realisees.asp',
 'http://www.finances.gouv.qc.ca/dette_du_quebec/fr/index.asp',
 'http://www.finances.gouv.qc.ca/fr/RI_OV_Obligations_vertes.asp',
 'http://www.finances.gouv.qc.ca/fr/Mesures_fiscales_Secteur_financier525.asp',
 'http://www.finances.gouv.qc.ca/fr/Mesures_fiscales_Secteur_financier545.asp',
 'http://www.finances.gouv.qc.ca/fr/Mesures_fiscales_Secteur_financier_et_entreprises523.asp',
 'http://www.finances.gouv.qc.ca/fr/Ministere675.asp',
 'ht

In [ ]:
import requests, re, ssl, os, sys
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry
from slugify import slugify

def getTextURL(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text
   
    soup = BeautifulSoup(html, "html.parser")
    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ").replace(u'\xa0', u' ')
    data = re.compile(r"\s+").sub(" ", data).strip() + "\n\n"
    
    return data

In [20]:
path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
for site in liste: 
    data = ""
    try: 
        text = getTextURL(site)
        data += text

        # encoding = liste[1][liste[0].index(site)] - pour prendre l'encodage qui a été extrait par le crawleur
        file_path = path + acteur + "/" + acteur + "-" + str(liste.index(site)) + ".txt"
        with open(file_path, 'w', encoding = "utf-8") as f:
            f.write(site + "\n")
            f.write(data)
        
    except Exception as e:
        print("ERROR " + " - " + site)
        print(e)

/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'avispublication.finances.gouv.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.quebec.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.quebec.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

## Combine .txt files into corpora
Réf : https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/

In [1]:
# Importer les données (fichiers .txt)
import os

acteur = "chu"
path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
corpus = []

# Change the directory
os.chdir(path + acteur + "/")
len(os.listdir())

6397

In [2]:
for file in os.listdir():
    if file.endswith(".txt"):
        file_path = path + acteur + "/" + file
        
        with open(file_path, 'r', encoding = "UTF-8") as f:
            data = f.readlines()
            corpus.append(data[1])

with open (path + acteur + "/" + acteur + '-corpus_FR.txt', 'w', encoding= 'UTF-8') as f: 
    for s in corpus:
        f.write(s) 
f.close()

## Combine whole FR/EN corpus together

In [1]:
path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
acteur = 'ramq'
corpus = ''

os.chdir(path + acteur)

In [2]:
for file in os.listdir():
    if file.endswith(".txt"):
        file_path = path + acteur + '/' + file
        
        with open(file_path, 'r', encoding = "UTF-8") as f:
            data = f.readlines()
            corpus += data[1] + ' '

with open (path + acteur + '/' + acteur + '_corpus.txt', 'w', encoding= 'UTF-8') as f: 
    f.write(corpus)
f.close()